In [1]:
import warnings, json, io, yaml

import pandas as pd
from contextlib import redirect_stderr
from decimal import Decimal

from utils import get_logger

In [2]:
def create_new_col(raw_df, key, value):
    if not any(col in value for col in raw_df.columns):
        raw_df[key] = ''.join(map(str, value))
    
    else:
        raw_df[key] = raw_df[value].astype(str).apply('-'.join, axis=1)

    return raw_df

In [3]:
logger = get_logger()

logger.info('initial log message')

2024-02-27 00:59:44,719|data_wrangling_logger|INFO|initial log message


In [6]:
# with open('crisp_take_home_config.json') as f:
#     config_dict = json.load(f)
    
# config_dict 

with open("config.yml", "r") as file:
    config_dict = yaml.safe_load(file)
    
config_dict

{'renamed_cols': {'Order Number': 'OrderId',
  'Product Number': 'ProductId',
  'Product Name': 'ProductName',
  'Count': 'Quantity'},
 'new_cols': {'Unit': ['kg'], 'OrderDate': ['Year', 'Month', 'Day']},
 'dtype_cols': {'int': ['OrderId'],
  'str': ['ProductId', 'ProductName', 'Unit'],
  'decimal': ['Quantity'],
  'datetime': ['OrderDate']},
 'str_dtype_cols_manipulation': {'proper_case': ['ProductName']}}

In [5]:
from contextlib import redirect_stderr, closing
import io

f = io.StringIO()

with redirect_stderr(f):
    raw_df = pd.read_csv('/Users/peterphyall/Documents/profdev/crisp_take_home/dummy_input_data_manual.csv', 
                         on_bad_lines='warn')

if f.getvalue():
    logger.warning(f"Reading in line - bad line(s): \n{f.getvalue()}")

2024-02-27 00:59:55,345|data_wrangling_logger|WARNING|Reading in line - bad line: 
Skipping line 5: expected 10 fields, saw 11
Skipping line 7: expected 10 fields, saw 11
Skipping line 10: expected 10 fields, saw 11
Skipping line 13: expected 10 fields, saw 11




In [ ]:
raw_df

In [ ]:
raw_df.info(verbose=True)

In [ ]:
for key, value in config_dict['dtype_cols'].items():
    print(key, value)

In [ ]:
raw_df

In [6]:
# 1) rename cols
raw_df = raw_df.rename(columns=config_dict['renamed_cols'])

# 2) create new cols
for key, value in config_dict['new_cols'].items():
    raw_df = create_new_col(raw_df, key, value)
    
# 3) dtype conversions
for key, value in config_dict['dtype_cols'].items():
    if 'int' in key or 'str' in key:
        raw_df[value] = raw_df[value].astype(key)
    
    elif 'datetime' in key:
        raw_df[value] = raw_df[value].apply(pd.to_datetime)

    elif 'decimal' in key:
        raw_df[value] = raw_df[value].apply(lambda x: x.str.replace(',', "")).apply(lambda x: x.apply(Decimal))

# 4) str dtype conversions
for key, value in config_dict['str_cols_conversion'].items():
    if 'proper_case' in key:
        raw_df[value] = raw_df[value].apply(lambda x: x.str.title())

raw_df = raw_df[['OrderId', 'OrderDate', 'ProductId', 'ProductName', 'Quantity', 'Unit']]

In [7]:
raw_df

,OrderId,OrderDate,ProductId,ProductName,Quantity,Unit
0,1000,2018-01-01,P-10001,Arugola,5250.50,kg
1,1001,2017-12-12,P-10002,Iceberg Lettuce,500.00,kg
2,1002,2017-12-12,P-10002,Iceberg Lettuce,500.00,kg
3,1004,2017-12-12,P-10002,Iceberg Lettuce,500.00,kg
4,1006,2017-12-12,P-10002,Iceberg Lettuce,500.00,kg
5,1007,2017-12-12,P-10002,Iceberg Lettuce,500.00,kg
6,1009,2017-12-12,P-10002,Iceberg Lettuce,500.00,kg
7,1010,2017-12-12,P-10002,Iceberg Lettuce,500.00,kg


In [ ]:
raw_df.info(verbose=True)

In [9]:
type(raw_df['Quantity'].iloc[0])

decimal.Decimal

### read in config.yml

In [ ]:
my_str = 'a string that needs formatting'

eval("my_str.")

In [5]:
import yaml

with open("config.yml", "r") as file:
    config_dict = yaml.safe_load(file)
    
config_dict

{'renamed_cols': {'Order Number': 'OrderId',
  'Product Number': 'ProductId',
  'Product Name': 'ProductName',
  'Count': 'Quantity'},
 'new_cols': {'Unit': ['kg'], 'OrderDate': ['Year', 'Month', 'Day']},
 'dtype_cols': {'int': ['OrderId'],
  'str': ['ProductId', 'ProductName', 'Unit'],
  'decimal': ['Quantity'],
  'datetime': ['OrderDate']},
 'str_dtype_cols_manipulation': {'proper_case': ['ProductName']}}